# Preprocessing

In [528]:
import pandas as pd
import os
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import rasterio
import matplotlib.pyplot as plt
import time
from rasterstats import zonal_stats
from osgeo import gdal
import numpy as np

In [529]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Mandal_boundaries_pre')
mandal_boundaries = gpd.read_file('mandal_boundaries.shp')
mandal_boundaries = mandal_boundaries.reset_index()

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\tsdm\\')
district_boundaries = gpd.read_file('District_Boundary.shp')
district_boundaries = district_boundaries.reset_index()

In [530]:
def read_tiffs(boundaries, stat:str, loc_month_begin:int, loc_month_end:int, loc_year_begin:int, loc_year_end:int):

    columns =['index', 'Mandal_Nam', 'Dist_Name', 'geometry']

    stat_df = boundaries[columns]
    stat_df.head()

    files = os.listdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\'+stat+'\\Rescaled')
 
    for j in files:
        
        os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\'+stat+ '\\Rescaled')
        file = rasterio.open(j, mode = 'r')
        file_array = file.read(1) 
        affine = file.transform

        time = j[loc_year_begin:loc_year_end] + '-' + j[loc_month_begin:loc_month_end] + '-01'

        stat_df[time] = 0
        
        stat_mean = zonal_stats(boundaries.geometry, file_array, affine = affine, geojson_out = True, nodata = file.nodata, all_touched = True)
        
        stat_lst = []
        for a in range(0, len(boundaries)):
            stat_lst.append(stat_mean[a]['properties']['mean'])

        stat_df[time] = stat_lst

        os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Model\\Preprocessed_Data')
        stat_df.to_csv(stat+'.csv')

    return(stat_df)

In [531]:
# This function rescales the pixel value such that we derive the physical values
def del_rescale(j, nodata_value_new, file_path):
    tiff_open = gdal.Open(file_path) # open the GeoTIFF we want to rescale
    band = tiff_open.GetRasterBand(1) # Select the band
    tiff_array = tiff_open.ReadAsArray() # Assign raster values to a numpy nd array

    nodata_value_old = 255
    nodata_value_old_2 = tiff_open.GetRasterBand(1).GetNoDataValue()

    tiff_array_new = np.where(tiff_array != nodata_value_old, tiff_array, nodata_value_new) # Rescale pixel values and change nodatavalue to -9999
    tiff_array_new = np.where(tiff_array_new != nodata_value_old_2, tiff_array_new, nodata_value_new)

    col = tiff_open.RasterXSize # number of columns
    rows = tiff_open.RasterYSize # number of rows
    driver = tiff_open.GetDriver()

    # Create a new raster to save the average-values
    new_raster = driver.Create(j, col, rows, 1, gdal.GDT_Float32)

    # Copy the properties of the original raster
    new_raster.SetGeoTransform(tiff_open.GetGeoTransform())
    new_raster.SetProjection(tiff_open.GetProjection())

    # Add the average values to newly created raster
    new_raster.GetRasterBand(1).WriteArray(tiff_array_new)
    new_raster.GetRasterBand(1).SetNoDataValue(nodata_value_new)

    # close the raster
    new_raster = None
    del new_raster

In [532]:
# This function rescales the pixel value such that we derive the physical values
def rescale(j, nodata_value_new, Scaling, Offset, file_path):
    tiff_open = gdal.Open(file_path) # open the GeoTIFF we want to rescale
    band = tiff_open.GetRasterBand(1) # Select the band
    tiff_array = tiff_open.ReadAsArray() # Assign raster values to a numpy nd array

    nodata_value_old = tiff_open.GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata

    tiff_array_new = np.where(tiff_array != nodata_value_old, tiff_array * Scaling + Offset, nodata_value_new) # Rescale pixel values and change nodatavalue to -9999

    col = tiff_open.RasterXSize # number of columns
    rows = tiff_open.RasterYSize # number of rows
    driver = tiff_open.GetDriver()

    # Create a new raster to save the average-values
    new_raster = driver.Create(j, col, rows, 1, gdal.GDT_Float32)

    # Copy the properties of the original raster
    new_raster.SetGeoTransform(tiff_open.GetGeoTransform())
    new_raster.SetProjection(tiff_open.GetProjection())

    # Add the average values to newly created raster
    new_raster.GetRasterBand(1).WriteArray(tiff_array_new)
    new_raster.GetRasterBand(1).SetNoDataValue(nodata_value_new)

    # close the raster
    new_raster = None
    del new_raster

## Rescale Ozon, NO2, and Soil Temperature

In [533]:
#First set all 255 values from NO2 to the following new data values
ndv = -9999

path = 'C:/Users/Jesse/OneDrive/Documenten/NO2/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/Jesse/OneDrive/Documenten/NO2/Changed/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        del_rescale(i, ndv, tiff_path)
    except:
        print(i)

#For NO2, we need to rescale from 0-254 to 0-1500
scaling = 1500/254
offset = 0
ndv = -9999

path = 'C:/Users/Jesse/OneDrive/Documenten/NO2/Changed/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/Jesse/OneDrive/Documenten/NO2/Rescaled/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

Changed
Rescaled


In [534]:
#First set all 255 values from Ozon to the following new data values
ndv = -9999

path = 'C:/Users/Jesse/OneDrive/Documenten/Ozon/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/Jesse/OneDrive/Documenten/Ozon/Changed/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        del_rescale(i, ndv, tiff_path)
    except:
        print(i)

scaling = 400/254
offset = 100
ndv = -9999

#For Ozon, we need to rescale from 0-254 to 100-500
path = 'C:/Users/Jesse/OneDrive/Documenten/Ozon/Changed/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/Jesse/OneDrive/Documenten/Ozon/Rescaled/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

AURA_OZONE_M_2017-12-01_rgb_1440x720.TIFF
Changed
Rescaled


In [ ]:
#First set all 255 values from Soil Temperature to the following new data values
ndv = -9999

path = 'C:/Users/Jesse/OneDrive/Documenten/Soil_Temperature/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/Jesse/OneDrive/Documenten/Soil_Temperature/Changed/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        del_rescale(i, ndv, tiff_path)
    except:
        print(i)

#For Soil Temperature, we need to rescale from 0-254 to -25 - 45
scaling = 70/254
offset = -25
ndv = -9999

path = 'C:/Users/Jesse/OneDrive/Documenten/Soil_Temperature/Changed/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/Jesse/OneDrive/Documenten/Soil_Temperature/Rescaled/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

Changed
Rescaled


## Read All Data Values from Tiff values of PM2.5, NO2, Ozon, Soil Temperature, and Soil Moisture

In [ ]:
stat_df_PM25 = read_tiffs(mandal_boundaries, 'PM2.5', -9, -7, -13, -9)

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a c

In [ ]:
stat_df_NO2 = read_tiffs(mandal_boundaries, 'NO2', -24, -22, -29, -25)

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a c

In [ ]:
stat_df_OZON = read_tiffs(mandal_boundaries, 'Ozon', -23, -21, -28, -24)

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a c

In [ ]:
stat_df_ST = read_tiffs(mandal_boundaries, 'Soil_Temperature', -24, -22, -29, -25)

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a c

In [ ]:
stat_df_ST_1km = read_tiffs(mandal_boundaries, 'Soil_Temperature_1km', -6, -4, -11, -7)

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a c

In [ ]:
stat_df_SSM = read_tiffs(mandal_boundaries, 'SSM', -6, -4, -11, -7)

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a c

## Crop Area Data

In [661]:
os.chdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\crop_area_data')

filenames = os.listdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\crop_area_data')
frames = []

for j in filenames:
    df = pd.read_csv(j)
    if j == 'mahabubabad_kharif_2016-17.csv':
        df.columns = ['year', 'season', 'districtName', 'crop', 'mandalName', 'normalAreaSown', 'actualAreaSown']
    else:
        df.columns = ['year', 'season', 'districtName', 'mandalName', 'crop','normalAreaSown', 'actualAreaSown']
   
    frames.append(df)  
        
result = pd.concat(frames)

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Model\\Preprocessed_Data')


In [662]:
result['Year'] = 0
result['year'] = result['year'].astype(str)
for i in range(0, len(result)):
    result['Year'].iloc[i] = result['year'].iloc[i][0:4]

del result['year']

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [663]:
result.columns = ['Season', 'Dist_Name', 'Mandal_Nam', 'Crop', 'NormalAreaSown', 'ActualAreaSown', 'Year']

In [664]:
#District Name Changes
result['Dist_Name'] = result['Dist_Name'].replace('Asifabad', 'Kumurambheem Asifabad')
result['Dist_Name'] = result['Dist_Name'].replace('Jagital', 'Jagtial')
result['Dist_Name'] = result['Dist_Name'].replace('Jagital', 'Jagtial')
result['Dist_Name'] = result['Dist_Name'].replace('Jangaon', 'Jangoan')
result['Dist_Name'] = result['Dist_Name'].replace('Kumuram Bheem', 'Kumurambheem Asifabad')
result['Dist_Name'] = result['Dist_Name'].replace('Medchal', 'Medchal_Malkajgiri')
result['Dist_Name'] = result['Dist_Name'].replace('Rajanna Siricilla', 'Rajanna Sircilla')
result['Dist_Name'] = result['Dist_Name'].replace('Warangal (Rural)', 'Warangal_Rural') 
result['Dist_Name'] = result['Dist_Name'].replace('Warangal (Urban)', 'Warangal_Urban') 

#Mandal Name Changes 
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Adilabad R', 'Adilabad Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Adilabad U', 'Adilabad Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gudihathnoor', 'Gudihathnur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bheempur', 'Bheempoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Utnoor', 'Utnur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jainath', 'Jainad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Srikonda', 'Sirikonda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bazarhatnoor', 'Bazarhathnoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sirpur-U', 'Sirpur_U')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Adilabad (Rural)', 'Adilabad Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Adilabad (Urban)', 'Adilabad Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Keramari', 'Kerameri')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dehegaon', 'Dahegaon')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Julurphad', 'Julurupad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Palwancha', 'Palvancha')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Aswaraopet', 'Aswaraopeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dammapet', 'Dammapeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mulkalpalli', 'Mulakalapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chunchupalli', 'Chunchupally') 
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Laxmidevipalli', 'Laxmidevipally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Julurpadu', 'Julurupad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tekulapally', 'Tekulapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mulkalapally', 'Mulakalapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Annapureddipalli', 'Annapureddipalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ieeza', 'Aiza')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dharur', 'Dharoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kaloor Timmanadoddi', 'Kaloor_Thimmandoddi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Undavelli', 'Undavelly')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Waddepalle', 'Waddepally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Itikyala', 'Itikyal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ieeja', 'Aiza')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ieja', 'Aiza')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Medipalli', 'Metpalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kathalapur', 'Kathlapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jagtial U', 'Jagtial')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jagtial R', 'Jagityal_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jagtial R ', 'Jagityal_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sarangapur', 'Sarangapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sirpur-T', 'Sirpur_T')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kowtala', 'Kouthala')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Manguru', 'Manuguru')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Burgampad', 'Burgampadu')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gattu', 'Ghattu')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tekulapalli', 'Tekulapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kothaguem', 'Kothagudem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gollapally', 'Gollapalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Beerupur', 'Beerpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Medipally', 'Medipalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Velgatoor', 'Velgatur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kodimyal', 'Kodimial')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bachannapeta', 'Bachannapet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jangaon', 'Jangoan')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Lingal Ghanpur', 'Lingalaghanpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Raghunathapalle', 'Ragunathpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Palakurthi', 'Palakurthy')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kodakondla', 'Kodakandla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Station Ghanpur', 'Ghanpur (Stn)')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Zaffargadh', 'Zaffergadh')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Lingala Ghanpur', 'Lingalaghanpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Raghunathpally', 'Ragunathpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Zaffarghad', 'Zaffergadh')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tharigopulla', 'Tharigoppula')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bhupalpalli', 'Bhupalpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Malhalrao', 'Malharrao')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Palimela', 'Palmela')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahamutharam', 'Mutharam_Mahadevpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ghanpur (Mulug)', 'Ghanapur_Mulug')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahamuttaram', 'Mutharam_Mahadevpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ghanpur (Mulg)', 'Ghanapur_Mulug')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Madnoor', 'Madnur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nizam sagar', 'Nizamsagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nagireddypet', 'Nagi_Reddypet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sadashivanagar', 'Sadashivnagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Biknoor', 'Bhiknoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Pedda kodapgal', 'Pedda Kodapga')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Naga_reddipet', 'Nagi_Reddypet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bachannpeta', 'Bachannapet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Devaruppala', 'Devaruppula')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kondakondla', 'Kodakandla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahadevpur', 'Mahadevpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ghanpur (Muug)', 'Ghanapur_Mulug')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bhiknur', 'Bhiknoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nasrullabad', 'Nasurullabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Pedda Kodapga', 'Pedda Kodapgal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rajamapet', 'Rajampet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sadasivanagar', 'Sadashivnagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chigurmamidi', 'Chigurumamidi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ellandhakunta', 'Ellandakunta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gannervaram', 'Ganneruvaram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jammikunta.', 'Jammikunta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Karimnagar (Rural)', 'Karimnagar_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Karimnagar Rural', 'Karimnagar_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Karimnagar Rural ', 'Karimnagar_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kothapalli', 'Kothapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Saidapur', 'V_Saidapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Veenavenka.', 'Veenavanka')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Venavanka', 'Veenavanka')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kamepalli', 'Kamepally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Khammam (R)', 'Khammam_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Khammam (U)', 'Khammam_Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Khammam (r)', 'Khammam_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Khammam (u)', 'Khammam_Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nelakondapalli', 'Nelakondapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sathupalli', 'Sathupally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tirumalaipalem', 'Tirumalayapalem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chintalamanepalli', 'Chintalamanepally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dahegoan', 'Dahegaon')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Lingapoor', 'Lingapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rebbana', 'Rebbena')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Wankdi', 'Wankidi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Danthalapally', 'Danthalapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Maripead', 'Maripeda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bhootpur', 'Bhoothpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Devarkadara', 'Devarkadra')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Hunwada', 'Hanwada')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('M nagar urban', 'Mahabubnagar_Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('M.nagar rural', 'Mahabubnagar_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahabubnagar (r)', 'Mahabubnagar_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahabubnagar ( r )', 'Mahabubnagar_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahabubnagar (u)', 'Mahabubnagar_Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahabubnagar(r)', 'Mahabubnagar_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mahabubnagar(u)', 'Mahabubnagar_Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Midgil', 'Midjil')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Musapet', 'Moosapet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Utkur', 'Utkoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bellampalli', 'Bellampalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bellampally', 'Bellampalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bhemini', 'Bheemini')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bhimaram', 'Bheemaram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bhimini', 'Bheemini')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dandepalli', 'Dandepalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dandepelly', 'Dandepalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Hazipur', 'Hajipur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kanepally', 'Kannepalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kannepally', 'Kannepalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kannepelly', 'Kannepalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kotapalli', 'Kotapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kotapally', 'Kotapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mancherial ', 'Mancherial')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mandamarry', 'Mandamarri')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nennal', 'Nennel')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemanpalli', 'Vemanpalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemanpally', 'Vemanpalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chilpched', 'Chilipched')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Haveli Ghanpur', 'Havelighanpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kowdipalle', 'Kowdipally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kowdipalli', 'Kowdipally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shankarampet (A)', 'Shankarampet_A')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shankarampet (R)', 'Shankarampet_R')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tupran', 'Toopran')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dundigal', 'Dundigal Gandimaisamma')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Qutbullapur', 'Quthbullapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bijinapally', 'Bijinapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bijinepalli', 'Bijinapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bijnepally', 'Bijinapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Charkonda', 'Charakonda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nagarkurnool', 'Nagar_Kurnool')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Padra', 'Padara')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Pedda kottapally', 'Peddakothapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Peddakothapally', 'Peddakothapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Pentlavelly', 'Pentlavelli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Telkapalli', 'Telkapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Telkapally', 'Telkapalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thimmajipet', 'Thimmajipeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Timmajipet', 'Thimmajipeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Uppununtala', '')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Uppununthala', 'Uppunuthala')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chinthakani', 'Chintakani')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Urakonda', 'Urkonda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Advidevulapally', 'Adavidevulapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Anumula', 'Anumula_Haliya')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chityala', 'Chityal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chityala ', 'Chityal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Damaracharala', 'Damaracherla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Damarcherla', 'Damaracherla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Deverkonda', 'Devarakonda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kangal', 'Kanagal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kattangoor', 'Kattangur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kethepalle', 'Kethepally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kethepalli', 'Kethepally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kondamallepally', 'Kondamallapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Madugulapaly', 'Madugulapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Miryalguda', 'Miryalaguda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Narketpalle', 'Narketpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Neredukommu', 'Neredugommu')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nidmanoor', 'Nidamanoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('P a pally', 'Peda Adisharla Palli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('P.A.Pally', 'Peda Adisharla Palli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Peddavooora', 'Peddavoora')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Saligouraram', 'Shaligouraram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shaligowraram', 'Shaligouraram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thipparthi', 'Tipparthy')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thipparthy', 'Tipparthy')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thirmalgirisager', 'Tirumalagiri_Sagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thirumalagiri Sagar', 'Tirumalagiri_Sagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dasturabad', 'Dasturabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dasturabad ', 'Dasturabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kaddam', 'Kaddampeddur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kaddam peddur', 'Kaddampeddur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mamada', 'Mamda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Narsapur G', 'Narsapur_G')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nirmal Urban', 'Nirmal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Armur', 'Armoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dharpalle', 'Dharpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dichpalle', 'Dichpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jakranpalle', 'Jakranpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kammarpalle', 'Kammarpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kotgiri', 'Kotagiri')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mortad', 'Morthad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nizamabad (Rural)', 'Nizamabad Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ranjal', 'Renjal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sirkonda', 'Sirikonda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vailpoor', 'Vailpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Yeda Palle', 'Yedapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Yeda_Palle', 'Yedapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Anthargaon', 'Anthergaon')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Anthargoan', 'Anthergaon')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Anthergoan', 'Anthergaon')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Eligedu', 'Eligaid')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Julapally', 'Julapalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mutharam', 'Mutharam_Manthani')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mutharam manthani', 'Mutharam_Manthani')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mutharam(manthani)', 'Mutharam_Manthani') 
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mutharam(mnt)', 'Mutharam_Manthani')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Peddapally', 'Peddapalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ramagundam (rural)', 'Ramagundam')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ramagundam(rural)', 'Ramagundam')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sulthanabad', 'Sultanabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Boinpally ', 'Boinpalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Boinpally', 'Boinpalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chandurthi', 'Chendurthi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chandurthy', 'Chendurthi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chendurthy', 'Chendurthi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ellanthakunta', 'Illanthakunta') 
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ellathkunta', 'Illanthakunta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ellathkunta ', 'Illanthakunta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gambhiraopet ', 'Gambhiraopet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Konaraopet ', 'Konaraopet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rudrangi ', 'Rudrangi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sircilla ', 'Sircilla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thangallapally', 'Thangallapalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Veernapally', 'Veernapalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemulawada ', 'Vemulawada')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemulawada (R)', 'Vemulawada Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemulawada (U)', 'Vemulawada')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemulawada (rural)', 'Vemulawada Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemulawada( rural)', 'Vemulawada Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vemulawada(urban)', 'Vemulawada')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Yellareddipet', 'Yellareddypet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Yellareddypet\t', 'Yellareddypet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Yellareddypet ', 'Yellareddypet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chowderigudem ', 'Chowdergudem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chwdarigudem', 'Chowdergudem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Faroornagar', 'Farooqnagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ibrahimpatan', 'Ibrahimpatnam')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kadtal', 'Kadthal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Keshampet', 'Keshampeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Maheshwaram', 'Maheswaram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Parigi', 'Pargi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rajendernagar ', 'Rajendranagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rajendra nagar', 'Rajendranagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Serilingampally', 'Serilingampalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shameerpet', 'Shamirpet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shankarpally', 'Shankarpalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shankerpally', 'Shankarpalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Talakondapalle', 'Talakondapally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('shabad', 'Shabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ameenpur ', 'Ameenpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gummadidala ', 'Gummadidala')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Hatnoora', 'Hathnoora')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jarasangam', 'Jharasangam')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kandi ', 'Kandi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kondapoor', 'Kondapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mogudampally ', 'Mogudampally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nagalgidda ', 'Nagalgidda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nagulgidda', 'Nagalgidda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Patancheru', 'Patancheruvu')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('R.C.Puram', 'Ramachandrapuram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('R.C.puram', 'Ramachandrapuram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('R.c.puram', 'Ramachandrapuram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ramchandrapuram ', 'Ramachandrapuram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Sirgapur', 'Sirgapoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Vatpally ', 'Vatpally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Zaheerabad', 'Zahirabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chinnakoduru.', 'Chinnakodur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Doultabad.', 'Doultabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Doulthabad', 'Doultabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dubbaka', 'Dubbak')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gajwel.', 'Gajwel.')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Raypole', 'Raipole')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Siddipet Rural', 'Siddipet_Rural')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Siddipet Urban', 'Siddipet_Urban')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Wargal.', 'Wargal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ananthagiri ', 'Ananthagiri')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chinthalapalem ', 'Chinthala palem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chinthalapalem', 'Chinthala palem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chivemla', 'Chivvemla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Garidepally', 'Garidepalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Huzurnagar', 'Huzur nagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jajireddygudem', 'Jajireddigudem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Maddirala ', 'Maddirala')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mattampally', 'Mattampalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nagaram ', 'Nagaram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Neredcherla', 'Nereducherla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nereducharla', 'Nereducherla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nutankal', 'Noothankal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nuthankal', 'Noothankal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Palakeedu ', 'Palakeedu')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Suryapeta', 'Suryapet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thirumalagiri', 'Tirumalagiri')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thungathurthi', 'Thungathurthy')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tirmalgiri', 'Tirumalagiri')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tungaturty', 'Thungathurthy')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bantaram', 'Bantwaram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bomraspet', 'Bommaraspeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bomraspeta', 'Bommaraspeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Doulatabad', 'Doulthabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Doultabad', 'Doulthabad')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kotpalle', 'Kotepally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kulkacherla', 'Kulkacharla')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Marpally', 'Marpalle')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Parigi', 'Pargi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Yalal', 'Yelal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Amarachintha', 'Amarchintha')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Peddamandadi', 'Peddamandaddi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nallabelly', 'Nallabelli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dugondi', 'Duggondi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rayaparthy', 'Raiparthy')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shyampet', 'Shayampet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bheemadevarapalle', 'Bheemadevarapalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bheemadevarapally', 'Bheemadevarapalli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Dharamasagar', 'Dharmasagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Elkathurthy', 'Elkathurthi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Inavole', 'Inavolu')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Inole', 'Inavolu')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kamalapoor', 'Kamalapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Khazipet', 'Khaazipet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Khilla_warangal', 'Khila Warangal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Killa-warangal	', 'Khila Warangal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Atmakur (m)', 'Athmakur (M)')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Atmakur(m)', 'Athmakur (M)')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('B.ramaram', 'Bommalaramaram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Bhongir', 'Bhongiri')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('M.turkapalle', 'Turkapalle_M')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('M.turkapally', 'Turkapalle_M')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Narayanapur', 'Narayanapoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Narayanpur', 'Narayanapoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Pochampalle', 'B.Pochampally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Pochampally', 'B.Pochampally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rajapet', 'Rajapeta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ramannapeta', 'Ramannapet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Thurkapally', 'Turkapalle_M')
result['Mandal_Nam'] = result['Mandal_Nam'].replace(' Mahadevpur', 'Mahadevpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Eturunagaram', 'Eturnagaram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mulug ', 'Mulug')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Tadvai', 'Tadvai (Sammakka Sarakka)')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Venkatapur ', 'Venkatapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('VENKATAPURAM', 'Venkatapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Venkatapuram', 'Venkatapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Venkatapuram ', 'Venkatapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Wajedu', 'Wazeed')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Kesavapatnam', 'Shankarapatnam')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chintalamanepally', 'Chintalmanepally')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Guduru', 'Gudur') 
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Narsimhulapeta', 'Narsimhulapet')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nellikkuduru', 'Nellikudur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Torrur', 'Thorrur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Cc kunta', 'Chinna Chinta Kunta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Cckunta', 'Chinna Chinta Kunta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chinna Chita Kunta', 'Chinna Chinta Kunta')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Damaragidda', 'Damargidda')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Narwa', 'Narva')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chowderigudem', 'Chowdergudem')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nandigam', 'Nandigama')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Qutbullapur', 'Quthbullapur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rajendernagar', 'Rajendranagar')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jagadevapur', 'Jagdevpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jagadevpur', 'Jagdevpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Jagadevpur ', 'Jagdevpur')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mulugu', 'Mulug')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mulugu.', 'Mulug')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Ramchandrapuram', 'Ramachandrapuram')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shankarampet-A', 'Shankarampet_A')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Shankarampet-R', 'Shankarampet_R')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Cheriyal', 'Cherial')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Gajwel.', 'Gajwel')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Komaravelly', 'Komuravelli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Komuravelly', 'Komuravelli')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Mirdoodi', 'Mirdoddi')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nanganoor', 'Nangnoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Nangunuru', 'Nangnoor')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Rayapole', 'Raipole')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Killa-warangal', 'Khila Warangal')
result['Mandal_Nam'] = result['Mandal_Nam'].replace('Chintalmanepally', 'Chintalamanepally')


unknown = result
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Bhadradri Kothagudem') & (unknown['Mandal_Nam']=='Kothaguda'), 'Mahabubabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jangoan') & (unknown['Mandal_Nam']=='Gundala'), 'Yadadri Bhuvanagiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Eturnagaram'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Govindaraopet'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Kannaigudem'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Mangapet'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Mulug'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Tadvai (Sammakka Sarakka)'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Venkatapur'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Jayashankar Bhupalpally') & (unknown['Mandal_Nam']=='Wazeed'), 'Mulugu' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Damargidda'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Dhanwada'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Kosgi'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Krishna'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Maddur'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Maganoor'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Makthal'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Marikal'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Narayanpet'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Narva'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Mahabubnagar') & (unknown['Mandal_Nam']=='Utkoor'), 'Narayanpet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Balanagar'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Bantwaram'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Basheerabad'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Dharoor'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Doma'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Gandeed'), 'Mahabubnagar' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Ghatkesar'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Keesara'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Kulkacharla'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Malkajgiri'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Marpalle'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Mechal'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Mominpet'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Nawabpet'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Pargi'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Peddemul'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Pudur'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Quthbullapur'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Shamirpet'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Tandur'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Uppal'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Vikarabad'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Yelal'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Alladurg'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Chegunta'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Chinnakodur'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Doulthabad'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Dubbak'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Gajwel'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Jagdevpur'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Kondapak'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Kowdipally'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Kulcharam'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Medak'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Mirdoddi'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Mulug'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Nangnoor'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Narsapur'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Papannapet'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Ramayampet'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Regode'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Shankarampet_A'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Shankarampet_R'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Shivampet'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Tekmal'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Toopran'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Wargal'), 'Siddipet' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Sangareddy') & (unknown['Mandal_Nam']=='Yeldurthy'), 'Medak' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Siddipet') & (unknown['Mandal_Nam']=='Doulthabad'), 'Vikarabad' , unknown['Dist_Name'])
unknown['Dist_Name'] = np.where((unknown['Dist_Name'] == 'Rangareddy') & (unknown['Mandal_Nam']=='Medchal'), 'Medchal_Malkajgiri' , unknown['Dist_Name'])


unknown['Mandal_Nam'] = np.where((unknown['Dist_Name'] == 'Jangoan') & (unknown['Mandal_Nam']=='Ghanpur'), 'Ghanpur (Stn)' , unknown['Mandal_Nam'])
unknown['Mandal_Nam'] = np.where((unknown['Dist_Name'] == 'Suryapet') & (unknown['Mandal_Nam']=='Athmakur '), 'Atmakur_S' , unknown['Mandal_Nam'])
unknown['Mandal_Nam'] = np.where((unknown['Dist_Name'] == 'Suryapet') & (unknown['Mandal_Nam']=='Atmakur'), 'Atmakur_S' , unknown['Mandal_Nam'])
unknown['Mandal_Nam'] = np.where((unknown['Dist_Name'] == 'Suryapet') & (unknown['Mandal_Nam']=='Atmakur '), 'Atmakur_S' , unknown['Mandal_Nam'])
unknown['Mandal_Nam'] = np.where((unknown['Dist_Name'] == 'Yadadri Bhuvanagiri') & (unknown['Mandal_Nam']=='Athmakur'), 'Athmakur (M)' , unknown['Mandal_Nam'])
unknown['Mandal_Nam'] = np.where((unknown['Dist_Name'] == 'Mahabubabad') & (unknown['Mandal_Nam']=='Kothagudem'), 'Kothaguda' , unknown['Mandal_Nam'])



result = unknown

In [665]:
crop_area = result 

crop_area['Crop'] = crop_area['Crop'].replace(' Cashewnut', 'Cashewnut')
crop_area['Crop'] = crop_area['Crop'].replace(' Tobacco', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace(' Turmeric', 'Turmeric')
crop_area['Crop'] = crop_area['Crop'].replace('Turemeric', 'Turmeric')
crop_area['Crop'] = crop_area['Crop'].replace('Turmaric', 'Turmeric')
crop_area['Crop'] = crop_area['Crop'].replace('Ajwam (Vamu)', 'Ajwain')
crop_area['Crop'] = crop_area['Crop'].replace('Ajwan(vamu)', 'Ajwain')
crop_area['Crop'] = crop_area['Crop'].replace('BAJRA', 'Bajra')
crop_area['Crop'] = crop_area['Crop'].replace('Beans', 'Bean')
crop_area['Crop'] = crop_area['Crop'].replace('Begal Gram', 'Bengal Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Bengal  Gram', 'Bengal Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Bengal gram', 'Bengal Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Bengalgram', 'Bengal Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Black  Gram', 'Black Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Black gram', 'Black Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Blackgram', 'Black Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Brijal', 'Brinjal')
crop_area['Crop'] = crop_area['Crop'].replace('Cabbabge', 'Cabbage')
crop_area['Crop'] = crop_area['Crop'].replace('Caster', 'Castor')
crop_area['Crop'] = crop_area['Crop'].replace('Cauli Flower', 'Califlower')
crop_area['Crop'] = crop_area['Crop'].replace('Cauliflower', 'Califlower')
crop_area['Crop'] = crop_area['Crop'].replace('Chililes', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Chilles', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Chilley', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Chilli', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Chillis', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Chillies & others', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Red chillies', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Green chillies', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Red Chillies', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Green Chillies', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Green Chiellies', 'Chillies')
crop_area['Crop'] = crop_area['Crop'].replace('Cow  Gram', 'Cow Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Cow Pea', 'Cow Gram')
crop_area['Crop'] = crop_area['Crop'].replace('cow pea', 'Cow Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Cow gram', 'Cow Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Cowgram', 'Cow Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Cowpea', 'Cow Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Green\rgram', 'Green Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Green gram', 'Green Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Greengram', 'Green Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Groudnut', 'Groundnut')
crop_area['Crop'] = crop_area['Crop'].replace('Ground nut', 'Groundnut')
crop_area['Crop'] = crop_area['Crop'].replace('Horsegram', 'Horse Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Onions', 'Onion')
crop_area['Crop'] = crop_area['Crop'].replace('RAGI', 'Ragi')
crop_area['Crop'] = crop_area['Crop'].replace('Red\rgram', 'Red Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Red  Gram', 'Red Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Red gram', 'Red Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Redgram', 'Red Gram')
crop_area['Crop'] = crop_area['Crop'].replace('Saffflower', 'Safflower')
crop_area['Crop'] = crop_area['Crop'].replace('Seasamum', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Seasmum', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Seasumam', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Seasumum', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Seasomum(Gingelly)', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Sesamum (Gingelly)', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Sesanum', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Sesumum', 'Sesamum')
crop_area['Crop'] = crop_area['Crop'].replace('Soya\rbean', 'Soyabean')
crop_area['Crop'] = crop_area['Crop'].replace('Soya bean', 'Soyabean')
crop_area['Crop'] = crop_area['Crop'].replace('Soyabeen', 'Soyabean')
crop_area['Crop'] = crop_area['Crop'].replace('Soyabin', 'Soyabean')
crop_area['Crop'] = crop_area['Crop'].replace('Soybean', 'Soyabean')
crop_area['Crop'] = crop_area['Crop'].replace('Sugar\rcane', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (Planted and Ratoon)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (Planted harvest)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (Planted)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (Ratoon harvest)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (Ratoon)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (planted harvest)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (planted sown)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (ratoon harvest)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane (ratoon sown)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane Planted (For harvest)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane planted', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane planted (harvest)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane planted (sown)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane ratoon', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugarcane ratoon (harvest)', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('Sugracane', 'Sugarcane')
crop_area['Crop'] = crop_area['Crop'].replace('SUN FLOWER', 'Sunflower')
crop_area['Crop'] = crop_area['Crop'].replace('Tabacco', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace('Tobacco Desi', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace('Tobaco', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace('Tobbocco', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace('Tobacco Desi', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace('Tobaco', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace('Tobbocco', 'Tobacco')
crop_area['Crop'] = crop_area['Crop'].replace('Tomatoes', 'Tomato')
crop_area['Crop'] = crop_area['Crop'].replace('Tomoto', 'Tomato')
crop_area['Crop'] = crop_area['Crop'].replace('Paddy Irrigated', 'Paddy')
crop_area['Crop'] = crop_area['Crop'].replace('Paddy Rainfed', 'Paddy')


In [666]:
crop_area['Season'] = crop_area['Season'].replace('Karif', 'Kharif')

In [667]:
foods = ['Maize', 'Paddy', 'Groundnut', 'Green Gram', 'Black Gram', 'Red Gram', 'Jowar', 'Sugarcane', 'Chillies', 'Cow Gram', 'Sesamum', 'Sunflower', 'Bengal Gram', 'Horse Gram', 'Cotton', 'Onion', 'Soyabean', 'Turmeric', 'Castor',
'Tobacco', 'Ragi', 'Wheat', 'Safflower', 'Potato', 'Coriander', 'Korra', 'Anumulu', 'Tomato', 'Rice']
len(foods)

29

In [668]:
crop_area['ActualAreaSown'] = crop_area['ActualAreaSown'].fillna(0)
crop_area['ActualAreaSown'] = crop_area['ActualAreaSown'].astype(int)
crop_area_selected  = crop_area[(crop_area['Crop'].isin(foods)) & (crop_area['ActualAreaSown']>0)]
crop_area_selected = crop_area_selected[(crop_area_selected['Year']=='2016') | (crop_area_selected['Year']=='2017') | (crop_area_selected['Year']=='2018')] 

In [669]:
crop_area =  crop_area_selected.merge(mandal_boundaries[['index', 'Dist_Name', 'Mandal_Nam', 'geometry']], on= ['Dist_Name', 'Mandal_Nam'], how = 'outer')

In [670]:
crop_area['index'] = crop_area['index'].fillna('unknown')

unknown_mandals = crop_area[crop_area['index']=='unknown']
known_mandals = crop_area[crop_area['index']!='unknown']

known_mandals['index'] = known_mandals['index'].astype(int)

<ipython-input-670-c419fa0e1274>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_mandals['index'] = known_mandals['index'].astype(int)


In [671]:
known_mandals['Crop'] = known_mandals['Crop'].fillna(9999)

unknown_vars = known_mandals[known_mandals['Crop']==9999]
crop_area = known_mandals[known_mandals['Crop']!=9999]

<ipython-input-671-406ba00213b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_mandals['Crop'] = known_mandals['Crop'].fillna(9999)


In [672]:
#Select only the maximum used Crop
max_crop_numbers = crop_area.groupby(['index', 'Dist_Name', 'Mandal_Nam', 'Season', 'Year'])['ActualAreaSown'].max().reset_index()
max_crop_numbers = max_crop_numbers.merge(crop_area[['index', 'Season', 'Year', 'Crop', 'ActualAreaSown']], on=['index', 'Season', 'Year', 'ActualAreaSown'], how='inner')
crops_total = pd.concat([max_crop_numbers, pd.get_dummies(max_crop_numbers['Crop'])], axis = 1)
foods_max = crops_total.columns[7:]
crops_dummies = crops_total.loc[:, ~crops_total.columns.isin(['Crop', 'NormalAreaSown', 'ActualAreaSown', 'geometry'])]

#Select All crops used
#crops_total = pd.concat([crop_area, pd.get_dummies(crop_area['Crop'])], axis = 1)
#crops_dummies = crops_total.groupby(['Dist_Name', 'Mandal_Nam', 'Season', 'Year', 'index'])[foods].sum().reset_index()

In [683]:
max_crop_numbers = crop_area.groupby(['index', 'Dist_Name', 'Mandal_Nam', 'Season', 'Year'])['ActualAreaSown'].max().reset_index()
max_crop_numbers = max_crop_numbers.merge(crop_area[['index', 'Season', 'Year', 'Crop', 'ActualAreaSown']], on=['index', 'Season', 'Year', 'ActualAreaSown'], how='inner')
max_crop_numbers[max_crop_numbers['index']==8]

,index,Dist_Name,Mandal_Nam,Season,Year,ActualAreaSown,Crop
46,8,Siddipet,Akkannapet,Kharif,2016,12280.0,Cotton
47,8,Siddipet,Akkannapet,Kharif,2017,5787.0,Cotton
48,8,Siddipet,Akkannapet,Kharif,2018,4252.0,Cotton
49,8,Siddipet,Akkannapet,Rabi,2016,5057.0,Paddy
50,8,Siddipet,Akkannapet,Rabi,2017,2275.0,Paddy
51,8,Siddipet,Akkannapet,Rabi,2018,210.0,Maize
52,8,Siddipet,Akkannapet,Rabi,2018,210.0,Groundnut


In [684]:
max_crop_numbers = crop_area.groupby(['index', 'Dist_Name', 'Mandal_Nam', 'Season', 'Year'])['ActualAreaSown'].max().reset_index()
max_crop_numbers[max_crop_numbers['index']==8]

,index,Dist_Name,Mandal_Nam,Season,Year,ActualAreaSown
46,8,Siddipet,Akkannapet,Kharif,2016,12280.0
47,8,Siddipet,Akkannapet,Kharif,2017,5787.0
48,8,Siddipet,Akkannapet,Kharif,2018,4252.0
49,8,Siddipet,Akkannapet,Rabi,2016,5057.0
50,8,Siddipet,Akkannapet,Rabi,2017,2275.0
51,8,Siddipet,Akkannapet,Rabi,2018,210.0


In [687]:
crop_area[(crop_area['index']==8) & (crop_area['Season']=='Rabi') & (crop_area['Year']=='2018')]

,Season,Dist_Name,Mandal_Nam,Crop,NormalAreaSown,ActualAreaSown,Year,index,geometry
16995,Rabi,Siddipet,Akkannapet,Paddy,1694.0,74.0,2018,8,"POLYGON ((79.17006 18.10883, 79.17029 18.10866..."
16996,Rabi,Siddipet,Akkannapet,Maize,1379.0,210.0,2018,8,"POLYGON ((79.17006 18.10883, 79.17029 18.10866..."
16997,Rabi,Siddipet,Akkannapet,Green Gram,11.0,7.0,2018,8,"POLYGON ((79.17006 18.10883, 79.17029 18.10866..."
16998,Rabi,Siddipet,Akkannapet,Black Gram,13.0,9.0,2018,8,"POLYGON ((79.17006 18.10883, 79.17029 18.10866..."
16999,Rabi,Siddipet,Akkannapet,Cow Gram,82.0,33.0,2018,8,"POLYGON ((79.17006 18.10883, 79.17029 18.10866..."
17000,Rabi,Siddipet,Akkannapet,Bengal Gram,30.0,5.0,2018,8,"POLYGON ((79.17006 18.10883, 79.17029 18.10866..."
17001,Rabi,Siddipet,Akkannapet,Groundnut,362.0,210.0,2018,8,"POLYGON ((79.17006 18.10883, 79.17029 18.10866..."


In [689]:
#Mandals are not in the dataframe at all
missing_mandals = pd.DataFrame(columns=['Dist_Name', 'Mandal_Nam', 'Season', 'Year', 'index'])

for i in range(0, len(unknown_vars)):
    for season in ['Kharif', 'Rabi']:
        for year in ['2016', '2017', '2018']:
            data = {'Dist_Name': unknown_vars['Dist_Name'].iloc[i], 'Mandal_Nam': unknown_vars['Mandal_Nam'].iloc[i], 'Season': season, 'Year':year, 'index': unknown_vars['index'].iloc[i]}
            missing_mandals = missing_mandals.append(data, ignore_index=True)

In [690]:
#Mandals that are in the dataframe but incomplete
counter = crops_dummies.pivot_table(index='index', 
                     columns=['Season', 'Year'], 
                     values='Dist_Name',
                     fill_value=0, 
                     aggfunc='count').unstack().reset_index()

non_complete = counter[counter[0]==0]

non_complete = non_complete.merge(mandal_boundaries[['Dist_Name', 'Mandal_Nam', 'index']], on='index')

In [691]:
crops_dummies = crops_dummies.merge(non_complete[['Season', 'Year', 'index', 'Dist_Name', 'Mandal_Nam']], on=['Season', 'Year', 'index', 'Dist_Name', 'Mandal_Nam'], how='outer')    
crops_dummies = crops_dummies.merge(missing_mandals[['Season', 'Year', 'index', 'Dist_Name', 'Mandal_Nam']], on=['Season', 'Year', 'index', 'Dist_Name', 'Mandal_Nam'], how='outer')    

In [692]:
crops_dummies['index'] = crops_dummies['index'].astype(int)

In [693]:
crops_dummies = crops_dummies.fillna('Unknown')

In [694]:
unknowns = crops_dummies[crops_dummies['Paddy']=='Unknown']
crops_dummies = crops_dummies[crops_dummies['Paddy']!='Unknown']

In [695]:
majorities_year = crop_area.groupby(['Dist_Name', 'Season', 'Year', 'Crop'])['ActualAreaSown'].sum().reset_index()
majorities_season = crop_area.groupby(['Dist_Name', 'Season', 'Crop'])['ActualAreaSown'].sum().reset_index()

In [696]:
for i in range (0, len(unknowns)):
    if unknowns['Dist_Name'].iloc[i] != 'Hyderabad':
        df_year = majorities_year[(majorities_year['Dist_Name']== unknowns['Dist_Name'].iloc[i]) & (majorities_year['Year'] == unknowns['Year'].iloc[i]) & (majorities_year['Season'] == unknowns['Season'].iloc[i])]
        df_season = majorities_season[(majorities_season['Dist_Name']== unknowns['Dist_Name'].iloc[i]) & (majorities_season['Season'] == unknowns['Season'].iloc[i])]
        if df_year.empty == False:
            df_sorted_year = df_year.sort_values(by='ActualAreaSown', ascending = False)
            unknowns[df_sorted_year['Crop'].iloc[0]].iloc[i] = 1
        else:
            df_sorted_season = df_season.sort_values(by='ActualAreaSown', ascending = False)
            unknowns[df_sorted_season['Crop'].iloc[0]].iloc[i] = 1
        

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [697]:
crops_dummies = pd.concat([crops_dummies, unknowns])
crops_dummies = crops_dummies.replace('Unknown', 0)

In [698]:
crops_dummies[foods_max] = crops_dummies[foods_max].astype(int)
#crops_dummies[foods] = crops_dummies[foods].astype(int)

In [700]:
crops_dummies['Kharif'] = 0
crops_dummies['Rabi'] = 0


for i in range(0, len(crops_dummies)):
    if (crops_dummies['Season'].iloc[i] == 'Rabi'):
        crops_dummies['Rabi'].iloc[i] = 1
    else:
        crops_dummies['Kharif'].iloc[i] = 1 

del crops_dummies['Season']

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [704]:
crops_dummies = crops_dummies.drop_duplicates()
crops_dummies = crops_dummies.drop_duplicates(['index', 'Year', 'Kharif', 'Rabi'])


In [705]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Model\\Preprocessed_Data')
crops_dummies.to_csv('crops_dummies.csv')

# Crop Area

In [427]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Mandal_boundaries_pre')

In [432]:
mandals = gpd.read_file('mandal_boundaries.shp')
mandals = mandals.reset_index()
mandals[['index', 'Mandal_Nam', 'Dist_Name', 'geometry']]

,index,Mandal_Nam,Dist_Name,geometry
0,0,Abdullapurmet,Rangareddy,"POLYGON ((78.59021 17.34506, 78.59009 17.34537..."
1,1,Achampet,Nagarkurnool,"POLYGON ((78.91204 16.54876, 78.91208 16.54716..."
2,2,Adavidevulapally,Nalgonda,"POLYGON ((79.52430 16.72522, 79.52860 16.72366..."
3,3,Addagudur,Yadadri Bhuvanagiri,"POLYGON ((79.36517 17.48429, 79.36700 17.48308..."
4,4,Addakal,Mahabubnagar,"POLYGON ((77.92377 16.58074, 77.92404 16.58049..."
...,...,...,...,...
587,587,Mosra,Nizamabad,"POLYGON ((78.03813 18.52915, 78.03453 18.53086..."
588,588,Chandur,Nizamabad,"POLYGON ((78.04060 18.49653, 78.03995 18.49642..."
589,589,Chowtakur,Sangareddy,"POLYGON ((78.06920 17.67324, 78.06918 17.67359..."
590,590,Masaipet,Medak,"POLYGON ((78.44551 17.94707, 78.44685 17.94670..."


In [447]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Landuse_Analysis')
tiff = '01-01-2019.tif'
lulc = rasterio.open(tiff, mode = 'r')
lulc_array = lulc.read(1) # landuse corresponding to each rasterpixel, so we extracted the pixel values from the raster
# affine: 1: corresponds to the width of each pixel, 2: row rotation, 3: x-coordinate of the upper left pixel, 4: column rotation, 5: height of each pixel, 6: y-coordinate of the upper left pixel
affine = lulc.transform

cmap = {1: 'Water', 2: 'Trees', 4: 'Flooded Vegetation', 5: 'Crops', 7: 'Built Area', 8: 'Bare Ground', 9: 'Snow/Ice', 10: 'Clouds', 11: 'Rangeland'}

test = zonal_stats(mandals.geometry, lulc_array, affine = affine, geojson_out = True, nodata = lulc.nodata, stats = 'majority', categorical=True, category_map = cmap, all_touched = True)


In [448]:
mandals['2018-12-31 Crops']=0
mandals['2018-12-31 Flooded Vegetation']=0

for i in range(0, len(test)):
    try:
        mandals['2018-12-31 Crops'].iloc[i] = test[i]['properties']['Crops'] 
    except:
        mandals['2018-12-31 Crops'].iloc[i] = 0
    try:
        mandals['2018-12-31 Flooded Vegetation'].iloc[i] = test[i]['properties']['Flooded Vegetation'] 
    except:
        mandals['2018-12-31 Flooded Vegetation'].iloc[i] = 0



c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [450]:
mandals['2016'] = mandals['2016-12-31 Crops'] + mandals['2016-12-31 Flooded Vegetation']
mandals['2017'] = mandals['2017-12-31 Crops'] + mandals['2017-12-31 Flooded Vegetation']
mandals['2018'] = mandals['2018-12-31 Crops'] + mandals['2018-12-31 Flooded Vegetation']

In [467]:
mandals = mandals[['index', 'Dist_Name', 'Mandal_Nam', '2016', '2017', '2018']]

In [468]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Model\\Preprocessed_Data')
mandals.to_csv('mandals_cropsarea.csv')

# FRP

In [457]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Classification_Fires')
#Select only the agricultural fires
fire_data_classified = gpd.read_file('fires_data_classified.geojson')
fire_data = fire_data_classified[fire_data_classified['agricultural'] == 1]

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Mandal_boundaries_pre')
mandal_boundaries = gpd.read_file('mandal_boundaries.shp')
mandal_boundaries = mandal_boundaries.reset_index()


In [464]:
#Create geodataframe from the data
geo_fire_data = gpd.GeoDataFrame(fire_data,geometry = fire_data.geometry, crs = {'init': 'epsg:4326'})

#Make sure the geometry columns are in the right format
geo_fire_data = geo_fire_data[['geometry', 'acq_date', 'fireID', 'frp']]
geo_fire_data['geometry'] = geo_fire_data['geometry'].to_crs(epsg = 4326)
boundaries = gpd.GeoDataFrame(mandal_boundaries,geometry = mandal_boundaries.geometry, crs = {'init': 'epsg:4326'}) 

#Load date into date format
geo_fire_data['acq_date'] =  pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data['year'] = (geo_fire_data['acq_date']).dt.year
geo_fire_data['month'] = (geo_fire_data['acq_date']).dt.month
geo_fire_data['day'] = (geo_fire_data['acq_date']).dt.day

#Selects the years we are interested in, depending on the input of the function
geo_fire_data = geo_fire_data[(geo_fire_data['acq_date'] >= '2015-01-01') & (geo_fire_data['acq_date'] < '2022-01-01')]

#Count all fires within a region given by the boundaries dataframe
fires_per_boundaries= gpd.sjoin(geo_fire_data, boundaries, how="inner")

#Create the right time format: We count per month
fires_per_boundaries['day'] = 1 
fires_per_boundaries['year'] = pd.Series(pd.to_numeric(fires_per_boundaries['year'], errors='coerce'), dtype='int64')
fires_per_boundaries['month'] = pd.Series(pd.to_numeric(fires_per_boundaries['month'], errors='coerce'), dtype='int64')
fires_per_boundaries['ModifiedDateTime'] = pd.to_datetime(fires_per_boundaries[['year', 'month', 'day']].astype('int64').astype('str'), yearfirst=True)

#Sum amount of fires per mandal per month per year make sure that if no fire happens at a specific time write a zero
fires_per_boundaries_count = fires_per_boundaries.groupby(['index', 'ModifiedDateTime'])['frp'].mean().unstack(fill_value=0).stack().reset_index()
fires_per_boundaries_count['FRP'] = fires_per_boundaries_count[0] 

counter = fires_per_boundaries.pivot_table(index='index', 
                     columns=['ModifiedDateTime'], 
                     values='frp',
                     fill_value=0, 
                     aggfunc='mean').unstack().reset_index()

#Delete column
del fires_per_boundaries_count[0]

#Make sure date is in the right format
fires_per_boundaries_count['ModifiedDateTime'] =  pd.to_datetime(fires_per_boundaries_count['ModifiedDateTime'])

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<ipython-input-464-8e87867c1aad>:2: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  geo_fire_data = gpd.GeoDataFrame(fire_data,geometry = fire_data.geometry, crs = {'init': 'epsg:4326'})
<ipython-input-464-8e87867c1aad>:7: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame

In [463]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Model\\Preprocessed_Data')
fires_per_boundaries_count.to_csv('frp_mandals.csv')